# Imports & Setup

In [13]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [14]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from nltk.stem.snowball import SnowballStemmer
import math

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  8 21:37 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [16]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [17]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/10 20:09:10 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [18]:
from inverted_index_gcp import InvertedIndex

In [19]:
spark

In [20]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'wikibucket208' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)


In [21]:
# Get some files from the bucket
def get_wiki_file_paths(all_wiki_files, num_files=3, all=False):
    # Dictionary to hold the file parts
    wiki_files_dict = {}

    # Regular expression to extract file number and part
    file_pattern = re.compile(r"multistream(\d+)_?(part\d+)?_preprocessed\.parquet")
    
    # Populate the dictionary with file numbers and their parts
    for file_path in all_wiki_files:
        match = file_pattern.search(file_path)
        if match:
            file_number = int(match.group(1))
            file_part = match.group(2) or "part1"  # Default to part1 if no part specified

            if file_number not in wiki_files_dict:
                wiki_files_dict[file_number] = []
            wiki_files_dict[file_number].append((file_part, file_path))

    # Sort the parts for each file number
    for file_number in wiki_files_dict:
        wiki_files_dict[file_number].sort()
    
    if(all):
        max_num = __builtin__.max(wiki_files_dict.keys())
        selected_files = list(sorted(wiki_files_dict.keys()))[:max_num]
    else:
        selected_files = list(sorted(wiki_files_dict.keys()))[:num_files]

    # Collect paths, ensuring all parts of a file number are included
    wiki_files_paths = []
    for file_number in selected_files:
        for _, file_path in wiki_files_dict[file_number]:
            wiki_files_paths.append(file_path)

    return wiki_files_paths


In [22]:
NUM_BUCKETS = 150
bucket_name = 'wikibucket208'
base_dir = f'{bucket_name}'

wiki_files_dir = f'{base_dir}/WikiFiles'

index_filename = f'{bucket_name}/InvertedIndex/'
inverted_dir = f'{base_dir}/InvertedIndex'
posting_locs_dir = f'{inverted_dir}/posting_locs'
pl_body_dir = f'{posting_locs_dir}/body'
pl_title_dir = f'{posting_locs_dir}/title'

In [23]:
all_files = False
num_wiki_files = 1   

wiki_files_paths = get_wiki_file_paths(paths,num_wiki_files, all_files)
parquetFile = spark.read.parquet(*wiki_files_paths)

In [24]:
# Count number of wiki pages
parquetFile.count()

21084

# Building an inverted index

Indexing Functions

In [25]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)


def token2bucket_id(token):           # Hashing function
    return int(_hash(token),16) % NUM_BUCKETS


def word_count(text, id):             # Tokenizing text + TF
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    stemmed_tokens = stem_tokens(tokens)

    tok_dic = {}
    non_stop_tokens = [token for token in tokens if token not in all_stopwords]
    for token in non_stop_tokens:
        tok_dic[token] = tok_dic.get(token, 0) + 1
    result = [(token, (id, tf)) for token, tf in tok_dic.items()]
    return result
   
    
def reduce_word_counts(unsorted_pl):  # Sorting posting list
    return sorted(unsorted_pl, key=lambda x: x[0])
    
    
def calculate_df(postings):           # Calculating DF
    return postings.map(lambda x: (x[0], len(x[1])))
    
    
def partition_postings_and_write(postings, save_dir):    # Writing posting locs to bucket
  def write_bucket_to_disk(bucket_id_and_postings, save_dir):
        # Pass both the tuple and the bucket_name to write_a_posting_list
    return  InvertedIndex.write_a_posting_list(bucket_id_and_postings, save_dir, bucket_name)

  postings_with_bucket_id = postings.map(lambda x: (token2bucket_id(x[0]), (x[0], x[1])))
  postings_grouped_by_bucket = postings_with_bucket_id.groupByKey().mapValues(list)
  bucket_ids = postings_grouped_by_bucket.map(
        lambda bucket_postings: write_bucket_to_disk((bucket_postings[0], list(bucket_postings[1])), save_dir)
  )
  return bucket_ids

In [26]:
# Tokenizer
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){,24}""", re.UNICODE)
def tokenize(text):
  return [token.group() for token in RE_WORD.finditer(text.lower())]

# Stop Words function
nltk_stop_words = set(stopwords.words('english'))

corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
all_stopwords = nltk_stop_words.union(corpus_stopwords)

def remove_stopwords(tokens):
    return [token for token in tokens if token not in all_stopwords]

In [27]:
# Stemmer Function
def stem_tokens(tokens, stem_model='porter'):
    if stem_model == 'porter':
        porter = PorterStemmer()
        stemmed_tokens = [porter.stem(token) for token in tokens]
    elif stem_model == 'snowball':
        snowball = SnowballStemmer(language='english')
        stemmed_tokens = [snowball.stem(token) for token in tokens]
    elif stem_model == None:
        return tokens
    else:
        raise ValueError("Unsupported stem model: {}".format(stem_model))
    return stemmed_tokens

In [28]:
# Hashing data to different buckets so the data will be split efficiently
from collections import defaultdict
def _hash(s):
  return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

def tokens2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

<h2 style="color: blue;">Indexing Body Posting Locs</h3>


In [29]:
    # Save Posting_locs Body
save_dir = 'InvertedIndex/posting_locs/body/'
doc_text_pairs = parquetFile.select("text", "id").rdd

    # word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)

    # filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
body_w2df_dict = w2df.collectAsMap()

    # partition posting lists and write out
_ = partition_postings_and_write(postings_filtered, save_dir).collect()

<h2 style="color: blue;">Indexing Body</h3>


In [30]:

save_dir = 'InvertedIndex'

    # Write Index values (tf/df) for Body
body_super_posting_locs = defaultdict(list)
body_posting_locs_prefix = 'InvertedIndex/posting_locs/body/'

for blob in client.list_blobs(bucket_name, prefix=body_posting_locs_prefix):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            body_super_posting_locs[k].extend(v)


    # Create inverted index instance
inverted = InvertedIndex()
    # Adding the posting locations dictionary to the inverted index
inverted.posting_locs = body_super_posting_locs
    # Add the token - df dictionary to the inverted index
inverted.df = body_w2df_dict
    # write the global stats out
inverted.write_index(save_dir, 'body_index', bucket_name)